# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Streamer Structrure for final Proyect** </center>
### <center> **Par de Foraneos** </center>


In [ ]:
import findspark
findspark.init()
import ta
from foraneos.utils_proyectofinal_copy import resample_and_aggregate
from foraneos.utils_proyectofinal_copy import SparkUtils as SpU

## Spark Session creation


In [ ]:
from pyspark.sql import SparkSession

SPARK_SERVER = {'Konrad': '2453c3db49e4',
                'Aaron' : 'a5ab6bdab4b3'}
KAFKA_SERVER = {'Konrad': '4c63f45c41b4:9093',
                'Aaron' : '69b1b3611d90:9093'}
current_user = 'Konrad'

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://{}:7077".format(SPARK_SERVER[current_user])) \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
    
sc = spark.sparkContext

spark.conf.set("spark.sql.shuffle.partitions", "5")

## Kafka Stream creation

In [ ]:
streamer_lines = []

for i in range(4):
    streamer_lines.append( spark \
                            .readStream \
                            .format("kafka") \
                            .option("kafka.bootstrap.servers", "{}".format(KAFKA_SERVER[current_user])) \
                            .option("subscribe", f"stock_topic{i}") \
                            .option("failOnDataLoss", "false")
                            .load()
    )


In [ ]:

result_schema = SpU.generate_schema([("timestamp", "timestamp" ), 
                                     ('company', 'string'),
                                              ("open", "float" ), 
                                              ("high", "float" ), 
                                              ("low", "float"),
                                              ("close", "float" )                                                                               
                                              ])




## Transform binary data into string

In [ ]:
from pyspark.sql.functions import col, split, window
from pyspark.sql.types import DoubleType, TimestampType
from pyspark.sql.functions import pandas_udf

streamer_df = []

for i in range(4):
             
    df = streamer_lines[i].withColumn("value_str", col("value").cast("string"))
    df = df.withColumn("split", split(col("value_str"), ","))
    df = df.withColumn("timestamp", col("split").getItem(0).cast(TimestampType())) \
           .withColumn("company", col("split").getItem(1)) \
           .withColumn("close", col("split").getItem(2).cast(DoubleType())) \
          .select("timestamp", "company","close")
          
    df.printSchema()
    custom_resampler = resample_and_aggregate(new_window=5)
    resampled_df = df.withWatermark("timestamp", "10 minutes") \
            .groupBy("company").applyInPandas(custom_resampler, schema=result_schema)
    #df.withWatermark("timestamp", "5 minutes").applyInPandas(custom_resampler, schema=result_schema)

    streamer_df.append(resampled_df)
    resampled_df.printSchema()



### Watermarking to handle late arrival events

### Sink configuration

In [ ]:
query = []

for i in range(4):
    query.append(
        streamer_df[i] \
        .writeStream \
        .outputMode("append") \
        .trigger(processingTime='120 seconds') \
        .format("parquet") \
        .option("path", f"/home/jovyan/notebooks/data/final_project_ParDeForaneos/output{i}/")
        .option("checkpointLocation", f"/home/jovyan/notebooks/data/final_project_ParDeForaneos/checkpoints/stock_topic{i}") \
        .start()
    )

In [ ]:
for i in range(4):
    query[i].stop()

In [ ]:
sc.stop()